In [ ]:
import pandas as pd

In [ ]:
df_raw = pd.read_csv(r"RAW DATA FILE.csv")
df_payer = pd.read_csv(r"PAYER DICTIONARY DATA.csv")

path = pd.ExcelFile(r'DICTIONARY 2.xlsx')
df_owner_mapping = pd.read_excel(path, 'Owner Mapping')

In [ ]:
new_row = []
alpha = 'A'
for i in range(len(df_raw.columns)):
    new_row.append(alpha)
    alpha = chr(ord(alpha) + 1)
df_raw.columns = new_row

In [ ]:
copy_raw_data = df_raw.iloc[:,17:24]
copy_raw_data.columns = ["Provider","Patient","Visit ID","Amount","Date","Age","Description"]

In [ ]:
op_path = r"OUTPUT.csv"
copy_raw_data.to_csv(op_path)

# we migrated the required data into a new file =>
# Now we work on the other required 3 columns using this new raw data

In [ ]:
df_new_raw = pd.read_csv(op_path)

In [ ]:
row_count_new_raw = sum(1 for row in df_new_raw["Provider"])
row_count_payer = sum(1 for row in df_payer["Visit ID"])

In [ ]:
def conversion(num):
    if "," in num:
        val = num.split(",")
        val = int(''.join(val))
        return val
    elif "," not in num:
        return int(num)

In [ ]:
payer_visit_id = df_payer["Visit ID"].tolist()
payer_payer_company = df_payer["Charge Payer Company"].tolist()

In [ ]:
temp_new_raw_visit_id = df_new_raw["Visit ID"].tolist()
new_raw_visit_id = []
for i in range(len(temp_new_raw_visit_id)):
    new_raw_visit_id.append(conversion(temp_new_raw_visit_id[i]))
    
temp_new_raw_desc = df_new_raw["Description"].tolist()
new_raw_desc = []
for i in temp_new_raw_desc:
    new_raw_desc.append(i.lower())

In [ ]:
temp_main_owner_map = df_owner_mapping["In-Limbo Error Description"].tolist()
main_owner_map = []
for i in temp_main_owner_map:
    main_owner_map.append(i.lower())
    
owner_val = df_owner_mapping["Error Order"].tolist()

In [ ]:
Payer_list = []
c = 0
for vid in new_raw_visit_id:
    if vid in payer_visit_id:
        idx = payer_visit_id.index(vid)
        payer_company = payer_payer_company[idx]
        Payer_list.append(payer_company)
        c+=1
    else:
        Payer_list.append("Visit ID not found")
        c+=1
print(c)

In [ ]:
Owner_list = []
c = 0
for desc in new_raw_desc:
    if desc in main_owner_map:
        idx = main_owner_map.index(desc)
        code = owner_val[idx]
        Owner_list.append(code)
        c+=1
    elif "invalid insurance type or no insurance associated" in desc.lower():
        Owner_list.append(2.2)
        c+=1
    else:
        Owner_list.append(float(-1))
        c+=1
print(c)

In [ ]:
min_list = []
id_owner = {}
c= 0
for i in range(len(new_raw_visit_id)):
    vid = new_raw_visit_id[i]
    if vid not in id_owner:
        id_owner[vid] = Owner_list[i]
    else:
        if id_owner[vid] > Owner_list[i]:
            id_owner[vid] = Owner_list[i]
        elif Owner_list[i] == float(-1):
            continue

for i in range(row_count_new_raw):
    vid = new_raw_visit_id[i]
    if vid in id_owner:
        min_list.append(id_owner[vid])
        c+=1
print(c)

In [ ]:
#deleting the rows with Patient Name = Cash, Mr. Unapplied
i = row_count_new_raw-1
x = i-1
idx = []
while i > 0:
    patient_name = df_new_raw["Patient"][i]
    
    if patient_name == "Cash, Mr. Unapplied":
        df_new_raw.drop(df_new_raw.index[i], inplace=True)
        idx.append(i)
        row_count_new_raw -= 1
        i = x
        x = i-1
    else:
        i-=1
        x = i-1
        
if df_new_raw["Patient"][0] == "Cash, Mr. Unapplied":
    df_new_raw.drop(df_new_raw.index[0], inplace=True)
    idx.append(i)
    row_count_new_raw -= 1
print(row_count_new_raw)

In [ ]:
df_new_raw.to_csv(op_path)

In [ ]:
#deleting the rows with Min = -1
i = len(min_list)-1
x = i-1
while i > 0:
    minval = min_list[i]
    
    if minval == float(-1) and i in idx:
#         df_op.drop(df_op.index[i], inplace=True)
        min_list.pop(i)
        Payer_list.pop(i)
        Owner_list.pop(i)
#         op_row_count -= 1
        i = x
        x = i-1
    else:
        i-=1
        x = i-1
        
if min_list[i] == float(-1) and 0 in idx:
#     df_op.drop(df_op.index[0], inplace=True)
    min_list.pop(0)
    Payer_list.pop(0)
    Owner_list.pop(0)
#     op_row_count -= 1

In [ ]:
for j in range(len(min_list)):
    if min_list[j] == float(-1):
        min_list[j] = "N/A"
# df_op["Min"] = min_list2
print(len(min_list))

In [ ]:
df_new_raw['Payer'] = Payer_list
df_new_raw['Owner'] = Owner_list
df_new_raw['Min'] = min_list
df_new_raw.to_csv(op_path)